In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from pathlib import Path

DATA_DIR='./emotions'#path to csv data file
wd = Path(DATA_DIR)

emotions = ['angry', 'fear', 'happy', 'sad']
sentiments = ['positive', 'negative']

In [23]:
import pandas as pd

def merge_df(working_dir: str, emotions):
    """
    Helper function to prepare the data by adding appropriate labels
    """

    dfs = []
    for cl in emotions:
        df = pd.read_csv(working_dir.joinpath(f'{cl}.csv')).drop(columns=['Unnamed: 1']).dropna()
        df['emotion'] = cl
        if cl == 'happy':
            df['sentiment'] = 'positive'
        else:
            df['sentiment'] = 'negative'
        dfs.append(df)
        
    return pd.concat(dfs)

In [44]:
from sklearn.preprocessing import LabelEncoder

#load and prepare dataframe
df = merge_df(wd, emotions)
df = df[df['Sentensi'].str.split().str.len().ge(2)]

#encoding the labels
sle, ele = LabelEncoder(),LabelEncoder()
for col,le in zip(['emotion', 'sentiment'], [ele, sle]):
    df[col] = le.fit_transform(df[col])

In [86]:
from text.data.vocab import Vocab

sentiment_voc = Vocab(data=df, text_col='Sentensi', morpheme_template='./df_morphs.txt')

  0%|          | 0/8600 [00:00<?, ?it/s]

In [87]:
from text.data.datasets import SAMDataset

data = SAMDataset(sentiment_voc, 'emotion', 'sentiment')

In [90]:
data[1]

([[[775],
   [561, 1592],
   [1511, 1902],
   [1361],
   [1092, 211],
   [821, 599],
   [73, 2023]]],
 0,
 0)

In [103]:
[[torch.as_tensor(sentiment_voc.label_encoder.encode(str(word))) for word in sentence.words] for i in range(2) for sentence in sentiment_voc.document[i]]

[[tensor(5544),
  tensor(3314),
  tensor(1781),
  tensor(3149),
  tensor(1996),
  tensor(8234),
  tensor(1708),
  tensor(6179)],
 [tensor(2858),
  tensor(1996),
  tensor(8594),
  tensor(5544),
  tensor(4849),
  tensor(2904),
  tensor(728)]]